In [59]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

df = pd.read_csv("./weatherAUS.csv")
df.head()

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from sklearn.model_selection import train_test_split
import tensorflow as tf

X = pd.read_pickle("./X.pk")
y = pd.read_pickle("./y.pk")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=66)

catgorical = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday']

X = pd.get_dummies(X, columns=catgorical, drop_first=True)

X_train_temp = X_train.copy()
X_test_temp = X_test.copy()

X_train_temp = pd.get_dummies(X_train_temp, columns=catgorical, drop_first=True)
X_test_temp = pd.get_dummies(X_test_temp, columns=catgorical, drop_first=True)

X_train = tf.constant(X_train_temp.values, dtype=tf.float32)
X_test = tf.constant(X_test_temp.values, dtype=tf.float32)
# del X_train_temp
# del X_test_temp

### 时序数据建模

#### 数据预处理

- 增加时间步，增加一个维度

> 类比于处理文本数据，如对文章进行分类，此数据一般是三个维度的，(label_num, seq_len, word_emb)
对于非文本类型的时序数据的维度是 (label_num, word_emb)，需要划分出多个样本，指定每个样本拥有的时间步 time_step
来将普通时序数据处理为 (label_num, time_step, word_emb)

In [12]:
# 处理 X_train_temp, X_test_temp

train = tf.data.Dataset.from_tensor_slices(tf.constant(X_train_temp, dtype=tf.float32))\
        .window(7, shift=1, drop_remainder=True).flat_map(lambda window:window.batch(32))

sample = train.take(1).as_numpy_iterator()
for i in sample:
    print(i.shape)

Cause: could not parse the source code:

        .window(7, shift=1, drop_remainder=True).flat_map(lambda window:window.batch(32))

This error may be avoided by creating the lambda in a standalone statement.

Cause: could not parse the source code:

        .window(7, shift=1, drop_remainder=True).flat_map(lambda window:window.batch(32))

This error may be avoided by creating the lambda in a standalone statement.

(7, 108)


In [60]:
from tensorflow.keras import *

# 滑动窗口
WINDOW_SIZE = 7

# 设定batch，为二维数据增加一个维度
def batch_dataset(dataset):
    return dataset.batch(WINDOW_SIZE, drop_remainder=True)

features = tf.data.Dataset.from_tensor_slices(X_train).window(WINDOW_SIZE, shift=1).flat_map(batch_dataset).shuffle(100)
labels = tf.data.Dataset.from_tensor_slices(tf.constant(y_train.values[WINDOW_SIZE:], dtype=tf.int32))

ds_train = tf.data.Dataset.zip((features, labels)).batch(32).cache()

In [61]:
tf.keras.backend.clear_session()

model = models.Sequential([
    layers.Input(shape=(None, X_train.shape[1]), dtype=tf.float32),
    layers.Bidirectional(layers.LSTM(32)),
    layers.Dense(32, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

history = model.fit(ds_train, epochs=5)

Epoch 1/5
3111/3111 [==============================] - 55s 18ms/step - loss: 0.7081 - accuracy: 0.7758
Epoch 2/5
3111/3111 [==============================] - 47s 15ms/step - loss: 0.6934 - accuracy: 0.7764
Epoch 3/5
3111/3111 [==============================] - 43s 14ms/step - loss: 0.6932 - accuracy: 0.7764
Epoch 4/5
3111/3111 [==============================] - 45s 14ms/step - loss: 0.6932 - accuracy: 0.7764
Epoch 5/5
3111/3111 [==============================] - 43s 14ms/step - loss: 0.6931 - accuracy: 0.7764


In [58]:
train_set = pd.get_dummies(X, columns=catgorical, drop_first=True)

WINDOW_SIZE = 7

features = tf.data.Dataset.from_tensor_slices(tf.constant(train_set.values, dtype=tf.float32)).window(WINDOW_SIZE, shift=1).flat_map(batch_dataset)
labels = tf.data.Dataset.from_tensor_slices(tf.constant(y.values[WINDOW_SIZE:], dtype=tf.int32))

ds_train = tf.data.Dataset.zip((features, labels)).batch(32).cache()

tf.keras.backend.clear_session()

model = models.Sequential([
    layers.Input(shape=(None, X_train.shape[1]), dtype=tf.float32),
    layers.LSTM(32),
    layers.Dense(32, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(1, activation="sigmoid")
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])


history = model.fit(ds_train, epochs=2) # 估计是梯度消失了

KeyError: "None of [Index(['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday'], dtype='object')] are in the [columns]"

In [31]:
# X_test 增加维度
X_test_tf = tf.expand_dims(X_test[-32:,:],axis = 0)
# X_test_tf = tf.data.Dataset.from_tensor_slices(X_test).window(WINDOW_SIZE, shift=1).flat_map(batch_dataset)

arr = model.predict(X_test_tf)

In [32]:
arr

array([[1.4728954e-06]], dtype=float32)

### TextCNN

In [78]:
cnn_model = models.Sequential([
#     layers.Input(shape=[7, X_train.shape[1]], dtype=tf.float32),
    layers.Conv1D(16, kernel_size= 5, activation = "relu", input_shape=(7, X_train.shape[1])),
    layers.MaxPool1D(),
    layers.Conv1D(128, kernel_size=2, activation = "relu"),
    layers.Flatten(),
    layers.Dense(1,activation = "sigmoid"),
])

cnn_model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])


cnn_model.fit(features, labels) # 训练集这玩意用 pandas 来处理绝逼能跑通

ValueError: Negative dimension size caused by subtracting 2 from 1 for 'conv1d_29/conv1d' (op: 'Conv2D') with input shapes: [?,1,1,16], [1,2,16,128].